In [1]:
import tensorflow as tf
import numpy as np, time, os

# 1. Данные
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train.reshape(-1, 784)/255.0, x_test.reshape(-1, 784)/255.0

# 2. Модель
model = tf.keras.Sequential([tf.keras.layers.Input((784,)), tf.keras.layers.Dense(128, activation='relu'), tf.keras.layers.Dense(10, activation='softmax')])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=3, verbose=0)
base_acc = model.evaluate(x_test, y_test, verbose=0)[1]

# 3. Конвертация Float32
conv_float = tf.lite.TFLiteConverter.from_keras_model(model)
model_float = conv_float.convert()
open('model_float.tflite', 'wb').write(model_float)

# 4. Конвертация Int8 (Квантование)
def rep_gen():
    for _ in range(100): yield [x_train[np.random.randint(0, len(x_train)):(np.random.randint(0, len(x_train))+1)].astype(np.float32)]

conv_quant = tf.lite.TFLiteConverter.from_keras_model(model)
conv_quant.optimizations = [tf.lite.Optimize.DEFAULT]
conv_quant.representative_dataset = rep_gen
conv_quant.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
model_quant = conv_quant.convert()
open('model_quant.tflite', 'wb').write(model_quant)

# 5. Сравнение
def benchmark(model_content, data, runs=50):
    interp = tf.lite.Interpreter(model_content=model_content); interp.allocate_tensors()
    inp, out = interp.get_input_details()[0], interp.get_output_details()[0]
    t = time.perf_counter()
    for _ in range(runs):
        interp.set_tensor(inp['index'], np.expand_dims(data[0], 0).astype(np.float32))
        interp.invoke()
    return time.perf_counter() - t

t_float, t_quant = benchmark(model_float, x_test), benchmark(model_quant, x_test)
s_float, s_quant = os.path.getsize('model_float.tflite'), os.path.getsize('model_quant.tflite')

print(f"Точность Keras: {base_acc:.4f}")
print(f"Размер: {s_float/1024:.2f} КБ -> {s_quant/1024:.2f} КБ (Экономия: {(1-s_quant/s_float)*100:.1f}%)")
print(f"Время инференса: {t_float:.4f}c -> {t_quant:.4f}c")

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
INFO:tensorflow:Assets written to: C:\Users\ELIZAV~1\AppData\Local\Temp\tmptuuxafm1\assets


INFO:tensorflow:Assets written to: C:\Users\ELIZAV~1\AppData\Local\Temp\tmptuuxafm1\assets


Saved artifact at 'C:\Users\ELIZAV~1\AppData\Local\Temp\tmptuuxafm1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 784), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  1777989271824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1777989277008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1777989276624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1777989277392: TensorSpec(shape=(), dtype=tf.resource, name=None)
INFO:tensorflow:Assets written to: C:\Users\ELIZAV~1\AppData\Local\Temp\tmp03zv09eg\assets


INFO:tensorflow:Assets written to: C:\Users\ELIZAV~1\AppData\Local\Temp\tmp03zv09eg\assets


Saved artifact at 'C:\Users\ELIZAV~1\AppData\Local\Temp\tmp03zv09eg'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 784), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  1777989271824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1777989277008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1777989276624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1777989277392: TensorSpec(shape=(), dtype=tf.resource, name=None)


c:\Users\Elizaveta\AppData\Local\Programs\Python\Python313\Lib\site-packages\tensorflow\lite\python\convert.py:863: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


Точность Keras: 0.9716
Размер: 399.21 КБ -> 105.23 КБ (Экономия: 73.6%)
Время инференса: 0.0023c -> 0.0022c


c:\Users\Elizaveta\AppData\Local\Programs\Python\Python313\Lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
